# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 2):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.09000000171363354


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
%load_ext autoreload
%autoreload 2

from collections import deque
from maddpg import MADDPG
from utils import ReplayBuffer
import torch

RAND_SEED = 42

agents = MADDPG(state_size=state_size, action_size=action_size, random_seed=RAND_SEED, n_agents=num_agents)

In [7]:
def run_episode(agent, train_mode=True):
    """Runs a single episode of the environment.
    Params
    ======
        agent (Agent): agent object from which to collect the experience
    """  
    env_info = env.reset(train_mode=train_mode)[brain_name]# reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = agent.act(states)                        # select an action (for each agent)
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += rewards                                  # update the score (for each agent)
        agent.step(states, actions, rewards, next_states, dones)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    return scores

In [8]:
def train_agent(agent, n_episodes=500, print_each=100, solve_window_size=100, solve_score=0.5):
    """Train a multi-agent DDPG model.
    Params
    ======
        agent (Agent): agent object to be trained
        n_episodes (int): number of episodes
        print_each (int): number of episodes to skip before printing current score
        solve_window_size (int): number of consecutive episodes to consider the environment solved
        solve_score (int): minimum average score to reach after solve_window_size
    """
    all_scores = []
    scores = []
    scores_window = deque(maxlen=solve_window_size)
    for i in range(1, n_episodes+1):        
        episode_scores = run_episode(agent)
        score = max(episode_scores)
        all_scores.append(episode_scores)
        scores.append(score)
        scores_window.append(score)
        avg_score = np.mean(scores_window)
        end = '\n' if (i%print_each == 0 or avg_score >= solve_score) else ''
        print("\rEpisode #{:3d}  |  "
              "Score: {:+6.2f}  |  "
              "Avg. Score: {:+6.2f}".format(i, score, avg_score),
              end=end
        )
        if avg_score >= solve_score and i >= solve_window_size:
            print('Environment solved in {} episodes!'.format(i))
            break

    # save model after all episodes
    #torch.save(agent.actor_local.state_dict(), 'model_actor.pt')
    #torch.save(agent.critic_local.state_dict(), 'model_critic.pt')
    #print('Model saved!')
    
    if np.mean(scores_window) < solve_score:
        print('Agent could not solve the environment in {} episodes'.format(n_episodes))
    
    return all_scores

In [ ]:
scores = train_agent(agents, n_episodes=3000, print_each=100)

Episode #100  |  Score:  +0.00  |  Avg. Score:  +0.01
Episode #200  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #300  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #400  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #500  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #600  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #700  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #800  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #900  |  Score:  +0.00  |  Avg. Score:  +0.00
Episode #975  |  Score:  +0.00  |  Avg. Score:  +0.00

In [ ]:
    def update(self, samples, agent_number):
        """update the critics and actors of all the agents """

        # need to transpose each element of the samples
        # to flip obs[parallel_agent][agent_number] to
        # obs[agent_number][parallel_agent]
        obs, obs_full, action, reward, next_obs, next_obs_full, done = samples

        obs_full = torch.stack(obs_full)
        next_obs_full = torch.stack(next_obs_full)

        agent = self.agents[agent_number]
        agent.critic_optimizer.zero_grad()

        # critic loss = batch mean of (y- Q(s,a) from target network)^2
        # y = reward of this timestep + discount * Q(st+1,at+1) from target network
        target_actions = self.target_act(next_obs)
        target_actions = torch.cat(target_actions, dim=1)

        target_critic_input = torch.cat((next_obs_full.t(), target_actions), dim=1)

        with torch.no_grad():
            q_next = agent.target_critic(target_critic_input)

        y = reward[agent_number].view(-1, 1) + GAMMA * q_next * (
                    1 - done[agent_number].view(-1, 1))
        action = torch.cat(action, dim=1)
        critic_input = torch.cat((obs_full.t(), action), dim=1)
        q = agent.critic(critic_input)

        huber_loss = torch.nn.SmoothL1Loss()
        critic_loss = huber_loss(q, y.detach())
        critic_loss.backward()
        # torch.nn.utils.clip_grad_norm_(agent.critic.parameters(), 0.5)
        agent.critic_optimizer.step()

        # update actor network using policy gradient
        agent.actor_optimizer.zero_grad()
        # make input to agent
        # detach the other agents to save computation
        # saves some time for computing derivative
        q_input = [self.agents[i].actor(ob) if i == agent_number \
                       else self.agents[i].actor(ob).detach()
                   for i, ob in enumerate(obs)]

        q_input = torch.cat(q_input, dim=1)
        # combine all the actions and observations for input to critic
        # many of the obs are redundant, and obs[1] contains all useful information already
        q_input2 = torch.cat((obs_full.t(), q_input), dim=1)

        # get the policy gradient
        actor_loss = -agent.critic(q_input2).mean()
        actor_loss.backward()
        # torch.nn.utils.clip_grad_norm_(agent.actor.parameters(),0.5)
        agent.actor_optimizer.step()

In [29]:
# obs, obs_full, action, reward, next_obs, next_obs_full, done = samples
# 
# obs_full = torch.stack(obs_full)
# next_obs_full = torch.stack(next_obs_full)
# 
# agent = self.agents[agent_number]
# agent.critic_optimizer.zero_grad()
agent_i = 0
states, actions, rewards, next_states, dones = memory.sample()
agent = agents.agents[agent_i]

In [62]:
states.dim()

3

In [55]:
states_full = states.reshape(BATCH_SIZE, -1)

In [ ]:
# if self.discrete_action:
#     trgt_vf_in = torch.cat((next_obs[agent_i],
#                             onehot_from_logits(
#                                 curr_agent.target_policy(
#                                     next_obs[agent_i]))),
#                            dim=1)
# else:
#     trgt_vf_in = torch.cat((next_obs[agent_i],
#                             curr_agent.target_policy(next_obs[agent_i])),
#                            dim=1)

In [101]:
agent_target_actions = agent.target_act(next_states[:, agent_i].numpy())
agent_target_actions = torch.Tensor(agent_target_actions)
print(agent_target_actions.shape)

torch.Size([32, 2])


In [98]:
q_next = agent.target_critic(next_states[:, agent_i], agent_target_actions)
q_next.shape

torch.Size([32, 1])

In [89]:
#target_value = (rews[agent_i].view(-1, 1) + self.gamma *
#                curr_agent.target_critic(trgt_vf_in) *
#                (1 - dones[agent_i].view(-1, 1)))
y = rewards[:, agent_i].view(-1, 1) + 0.99 * q_next * (1 - dones[:, agent_i].view(-1, 1))

In [92]:
# vf_in = torch.cat((obs[agent_i], acs[agent_i]), dim=1)
# actual_value = curr_agent.critic(vf_in)
actual_value = agent.critic(states[:, agent_i], actions[:, agent_i])
actual_value.shape

torch.Size([32, 1])

In [93]:
huber_loss = torch.nn.SmoothL1Loss()
critic_loss = huber_loss(actual_value, y.detach())
critic_loss.backward()